In [1]:
#import packages
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nibabel as nib
import os 
import sys
sys.path.append('../scripts')
import io_meld as io

# Intra Z calculation 

This script processes cortical features for a list of subjects using neuroimaging data stored in `.mgh` format. It normalises the features by calculating intra Z-scores for both hemispheres (left and right) and saves the processed data for each feature and subject. 

## Key Steps:

1. **Define Subject Directory and Features**:
    - The path to the subject data is defined as `subject_dir`.
    - A list of features (`feature_names`) to be processed is created, including features such as `gm_T1_0.75`, `thickness`, `curv`, and more.

2. **Subject List**:
    - The list of subjects is read from a file (`subject_list_path`). Each line in the file represents a subject ID.

3. **Load Cortex Mask**:
    - The cortex mask is loaded from a FreeSurfer label file (`cortex_file`) using `nibabel`, which specifies which parts of the brain are considered the cortex.

4. **Load Feature Data Function**:
    - The `load_feature_data` function loads left and right hemisphere feature data for each subject from the `.mgh` files in the `surf_meld` directory. The filenames are in the format `lh.on_lh.<feature>.mgh` and `rh.on_lh.<feature>.mgh`.

5. **Processing Each Subject**:
    - For each subject and feature:
        - The script loads the feature data for both hemispheres (left and right).
        - It creates full-size arrays filled with `NaN` values and assigns the feature data for the cortex regions using the loaded cortex mask.
        - It concatenates the left and right hemisphere data and computes the mean (`mu_sub`) and standard deviation (`std_sub`) across the cortex, ignoring `NaN` values.
        - Z-scores are calculated for the left and right hemisphere feature data using the formula:

          \[
          Z_{score} = \frac{(data - \mu)}{\sigma}
          \]

6. **Saving Z-scores**:
    - A template `.mgh` file is loaded (either from the left or right hemisphere) to extract the affine transformation and header information needed to save the new Z-score data.
    - The Z-score data for each hemisphere is saved to new `.mgh` files in the format `lh.intra_z.<feature>.mgh` and `rh.intra_z.<feature>.mgh`.

7. **Logging**:
    - A message is printed to confirm that the Z-score data has been saved for each subject and feature.



In [5]:

# Define the subject directory and feature names
subject_dir = 'path to subject directory '
feature_names = ['gm_T1_0.75', 'gm_T1_0.5', 'gm_T1_0.25', 'gm_T1_0', 'wm_T1_0.5', 'wm_T1_1', 'thickness', 'w-g.pct', 'pial.k_filtered.sm20', 'curv', 'sulc']
# Path to the file containing the list of subjects
subject_list_path = 'pathtosubjectlist.txt'
# Read the list of subjects from the file
with open(subject_list_path, 'r') as file:
    subjects = [line.strip() for line in file if line.strip()]
# Path to the cortex file
cortex_file = 'pathto/fsaverage_sym/label/lh.cortex.label'
# Load cortex mask
cortex = nib.freesurfer.read_label(cortex_file)  # Using nibabel to read Freesurfer label files directly
# Function to load feature data
def load_feature_data(subject, feature):
    lh_file = os.path.join(subject_dir, subject, 'xhemi', 'surf_meld', f'lh.on_lh.{feature}.mgh')
    rh_file = os.path.join(subject_dir, subject, 'xhemi', 'surf_meld', f'rh.on_lh.{feature}.mgh')
    lh_data = nib.load(lh_file).get_fdata()
    rh_data = nib.load(rh_file).get_fdata()
    return lh_data, rh_data
# Process each subject
for subject in subjects:
    for feature in feature_names:
        # Load feature data
        lh_feature, rh_feature = load_feature_data(subject, feature)
        # Create full-size arrays filled with NaNs
        lh_full = np.full(lh_feature.shape, np.nan)
        rh_full = np.full(rh_feature.shape, np.nan)
        # Assign cortex values to the full-size arrays
        lh_full[cortex] = lh_feature[cortex]
        rh_full[cortex] = rh_feature[cortex]
        # Compute mean and standard deviation ignoring NaNs
        concatenated_features = np.concatenate([lh_full[cortex], rh_full[cortex]])
        mu_sub = np.nanmean(concatenated_features)
        std_sub = np.nanstd(concatenated_features)
        # Normalize the features
        z_lh_feature = (lh_full - mu_sub) / std_sub
        z_rh_feature = (rh_full - mu_sub) / std_sub
        # Load one of the original MGH files to use as a template for saving
        template_mgh_file = os.path.join(subject_dir, subject, 'xhemi', 'surf_meld', f'lh.on_lh.{feature}.mgh')
        if not os.path.exists(template_mgh_file):
            template_mgh_file = os.path.join(subject_dir, subject, 'xhemi', 'surf_meld', f'rh.on_lh.{feature}.mgh')
        template_mgh = nib.load(template_mgh_file)
        template_affine = template_mgh.affine
        template_header = template_mgh.header
        # Create new MGHImage objects for Z-scores
        z_lh_img = nib.MGHImage(z_lh_feature, affine=template_affine, header=template_header)
        z_rh_img = nib.MGHImage(z_rh_feature, affine=template_affine, header=template_header)
        # Define output paths
        lh_outfile = os.path.join(subject_dir, subject, f'lh.intra_z.{feature}.mgh')
        rh_outfile = os.path.join(subject_dir, subject, f'rh.intra_z.{feature}.mgh')
        # Save the Z-score data to MGH files
        nib.save(z_lh_img, lh_outfile)
        nib.save(z_rh_img, rh_outfile)
        print(f"Z-score data for {subject} - {feature} has been saved to '{lh_outfile}' and '{rh_outfile}'.")
        

        

Z-score data for MELD_H1_C_003 - gm_T1_0.75 has been saved to '/Users/pierceburr/Documents/Documents/UCL_dissertation/meld/output/fastsurfer/MELD_H1_C_003/lh.intra_z.gm_T1_0.75.mgh' and '/Users/pierceburr/Documents/Documents/UCL_dissertation/meld/output/fastsurfer/MELD_H1_C_003/rh.intra_z.gm_T1_0.75.mgh'.
Z-score data for MELD_H1_C_003 - gm_T1_0.5 has been saved to '/Users/pierceburr/Documents/Documents/UCL_dissertation/meld/output/fastsurfer/MELD_H1_C_003/lh.intra_z.gm_T1_0.5.mgh' and '/Users/pierceburr/Documents/Documents/UCL_dissertation/meld/output/fastsurfer/MELD_H1_C_003/rh.intra_z.gm_T1_0.5.mgh'.
Z-score data for MELD_H1_C_003 - gm_T1_0.25 has been saved to '/Users/pierceburr/Documents/Documents/UCL_dissertation/meld/output/fastsurfer/MELD_H1_C_003/lh.intra_z.gm_T1_0.25.mgh' and '/Users/pierceburr/Documents/Documents/UCL_dissertation/meld/output/fastsurfer/MELD_H1_C_003/rh.intra_z.gm_T1_0.25.mgh'.
Z-score data for MELD_H1_C_003 - gm_T1_0 has been saved to '/Users/pierceburr/Docu

# Intersubject Normalisation

This script processes cortical features of neuroimaging data from multiple subjects, performing intersubject normalisation. The process involves computing cohort-level statistics (mean and standard deviation) for various cortical features, followed by the application of these statistics to normalise individual subjects' data.

The purpose of this step is to normalise the data by cortical region. 

## Key Steps:

### 1. Define Feature Names and Output Directory:
- **Features to be processed:** A list of cortical features (`feature_names`) such as `gm_T1_0.75`, `thickness`, `curv`, etc.
- **Output Directory:** The path where the processed control subject data will be saved is defined (`output_dir`). The directory is created if it does not already exist.

### 2. Read Subject List:
- The script reads a list of subjects from a specified file (`subject_list_path`), where each subject ID is read from a line in the file.

### 3. Function to Load Feature Data for Subjects:
The function `load_feature_data()` loads feature data for each subject based on lesion files:
- **Lesion files:** `lh.lesion.nii` for the left hemisphere, and `rh.lesion.nii` for the right hemisphere, indicating which hemisphere has a lesion.
- **Contralateral hemisphere data:** If a lesion exists in one hemisphere, the feature data from the contralateral (opposite) hemisphere is loaded. 
- The data from the contralateral hemisphere across all subjects is stacked into a 3D array for further processing.

### 4. Calculate Mean and Standard Deviation for Each Feature:
For each feature in the `feature_names` list:
- **Mean and Standard Deviation:** The script computes the mean (`mean_intensity`) and standard deviation (`sd_intensity`) for each feature across all subjects, while ignoring `NaN` values.
- **Saving cohort statistics:** The mean and SD are saved as new `.mgh` files in the output directory using the filenames `mu.intra_z.<feature>.mgh` and `std.intra_z.<feature>.mgh`.

### 5. Intersubject Normalization:
Once cohort-level mean and SD values are computed, the script performs intersubject normalization for each subject:
- **Loading cohort statistics:** The mean and SD files are loaded from the output directory.
- **Normalization process:** For each feature, the script adjusts the subject’s intra-Z score data by normalizing it against the cohort mean and SD using the formula:

  \[
  Z_{inter} = \frac{(Z_{intra} - \mu_{cohort})}{\sigma_{cohort}}
  \]

- **Saving normalized data:** The intersubject normalized data is saved as new `.mgh` files using filenames `lh.inter_z.intra_z.<feature>.mgh` and `rh.inter_z.intra_z.<feature>.mgh` for the left and right hemispheres respectively.

### 6. Logging:
After processing each feature for each subject, the script logs a message indicating that the intersubject normalized data has been saved.




In [4]:

# Directory to save control subject data
output_dir = 'path/MELD_control'
os.makedirs(output_dir, exist_ok=True)

# Read the list of subjects from the file
with open(subject_list_path, 'r') as file:
    subjects = [line.strip() for line in file if line.strip()]

# Function to load MGH files for a specific feature
def load_feature_data(subject_dir, subjects, feature_name):
    contralateral_data_list = []
    
    for subject in subjects:
        lh_lesion_file = os.path.join(subject_dir, subject, 'lh.lesion.nii')
        rh_lesion_file = os.path.join(subject_dir, subject, 'rh.lesion.nii')
        
        if os.path.exists(lh_lesion_file):
            contralateral_feature_file = os.path.join(subject_dir, subject, f'rh.intra_z.{feature_name}.mgh')
        elif os.path.exists(rh_lesion_file):
            contralateral_feature_file = os.path.join(subject_dir, subject, f'lh.intra_z.{feature_name}.mgh')
        else:
            print(f"Warning: No lesion mask found for subject {subject}. Skipping.")
            continue
        
        if os.path.exists(contralateral_feature_file):
            contralateral_mgh_data = nib.load(contralateral_feature_file).get_fdata()
            contralateral_data_list.append(contralateral_mgh_data)
        else:
            print(f"Warning: {contralateral_feature_file} does not exist and will be skipped.")
    
    if contralateral_data_list:
        return np.stack(contralateral_data_list, axis=-1)
    else:
        return None

# Process each feature for intersubject normalization
for feature in feature_names:
    # Load the data for the current feature
    data_array = load_feature_data(subject_dir, subjects, feature)
    
    if data_array is not None:
        # Calculate mean and standard deviation along the subjects axis, ignoring NaNs
        mean_intensity = np.nanmean(data_array, axis=-1)
        sd_intensity = np.nanstd(data_array, axis=-1)
        
        # Load one of the original MGH files to use as a template for saving
        template_mgh_file = os.path.join(subject_dir, subjects[0], f'lh.intra_z.{feature}.mgh')
        if not os.path.exists(template_mgh_file):
            template_mgh_file = os.path.join(subject_dir, subjects[0], f'rh.intra_z.{feature}.mgh')
        
        template_mgh = nib.load(template_mgh_file)
        template_affine = template_mgh.affine
        template_header = template_mgh.header

        # Create new MGHImage objects for mean and SD
        mean_img = nib.MGHImage(mean_intensity, affine=template_affine, header=template_header)
        sd_img = nib.MGHImage(sd_intensity, affine=template_affine, header=template_header)

        # Define output paths
        mean_output_path = os.path.join(output_dir, f'mu.intra_z.{feature}.mgh')
        sd_output_path = os.path.join(output_dir, f'std.intra_z.{feature}.mgh')

        # Save the mean and SD data to MGH files
        nib.save(mean_img, mean_output_path)
        nib.save(sd_img, sd_output_path)

        print(f"Control subject data for {feature} has been saved to '{mean_output_path}' and '{sd_output_path}'.")
    else:
        print(f"No data found for feature {feature} across all subjects.")

# Adjust each patient's intra_z feature using the template mu and std
for subject in subjects:
    for feature in feature_names:
        # Paths to the mu and std files
        mu_file = os.path.join(output_dir, f'mu.intra_z.{feature}.mgh')
        std_file = os.path.join(output_dir, f'std.intra_z.{feature}.mgh')
        
        # Load the template mu and std
        mu_intra_cohort = nib.load(mu_file).get_fdata().squeeze()
        std_intra_cohort = nib.load(std_file).get_fdata().squeeze()
        
        # Paths to the subject's intra_z files
        lh_intra_file = os.path.join(subject_dir, subject, f'lh.intra_z.{feature}.mgh')
        rh_intra_file = os.path.join(subject_dir, subject, f'rh.intra_z.{feature}.mgh')
        
        if os.path.exists(lh_intra_file) and os.path.exists(rh_intra_file):
            # Load the subject's intra_z files
            lh_intra_sub = nib.load(lh_intra_file).get_fdata().squeeze()
            rh_intra_sub = nib.load(rh_intra_file).get_fdata().squeeze()
            
            # Z-scoring with NaNs ignored
            lh_inter_intra_sub = (lh_intra_sub - mu_intra_cohort) / std_intra_cohort
            rh_inter_intra_sub = (rh_intra_sub - mu_intra_cohort) / std_intra_cohort
            
            # Create new MGHImage objects for intersubject normalized data
            lh_inter_img = nib.MGHImage(lh_inter_intra_sub, affine=template_affine, header=template_header)
            rh_inter_img = nib.MGHImage(rh_inter_intra_sub, affine=template_affine, header=template_header)
            
            # Define output paths
            lh_inter_output_path = os.path.join(subject_dir, subject, f'lh.inter_z.intra_z.{feature}.mgh')
            rh_inter_output_path = os.path.join(subject_dir, subject, f'rh.inter_z.intra_z.{feature}.mgh')
            
            # Save the intersubject normalized data to MGH files
            nib.save(lh_inter_img, lh_inter_output_path)
            nib.save(rh_inter_img, rh_inter_output_path)
            
            print(f"Intersubject normalised data for {subject} - {feature} has been saved to '{lh_inter_output_path}' and '{rh_inter_output_path}'.")


/var/folders/q8/cxyrc7dd0wnbxymt43rljx6c0000gn/T/ipykernel_67428/2596850832.py:43: RuntimeWarning: Mean of empty slice
  mean_intensity = np.nanmean(data_array, axis=-1)


Control subject data for gm_T1_0.75 has been saved to '/Users/pierceburr/Documents/Documents/UCL_dissertation/meld/output/fastsurfer/MELD_control/mu.intra_z.gm_T1_0.75.mgh' and '/Users/pierceburr/Documents/Documents/UCL_dissertation/meld/output/fastsurfer/MELD_control/std.intra_z.gm_T1_0.75.mgh'.
Control subject data for gm_T1_0.5 has been saved to '/Users/pierceburr/Documents/Documents/UCL_dissertation/meld/output/fastsurfer/MELD_control/mu.intra_z.gm_T1_0.5.mgh' and '/Users/pierceburr/Documents/Documents/UCL_dissertation/meld/output/fastsurfer/MELD_control/std.intra_z.gm_T1_0.5.mgh'.
Control subject data for gm_T1_0.25 has been saved to '/Users/pierceburr/Documents/Documents/UCL_dissertation/meld/output/fastsurfer/MELD_control/mu.intra_z.gm_T1_0.25.mgh' and '/Users/pierceburr/Documents/Documents/UCL_dissertation/meld/output/fastsurfer/MELD_control/std.intra_z.gm_T1_0.25.mgh'.
Control subject data for gm_T1_0 has been saved to '/Users/pierceburr/Documents/Documents/UCL_dissertation/me

This code processes neuroimaging data for multiple subjects to calculate lesion and contralateral hemisphere z-scores based on specified features, saving the results in a CSV file. Here’s an outline of the workflow:

1. **Read Subject List**: Reads a list of subjects from a text file.

2. **Calculate Z-scores**: For each subject and feature:
   - Defines paths to the z-score and lesion mask files for both hemispheres.
   - Loads the z-score data and lesion mask, checks for errors, and verifies that their dimensions match.
   - If a valid lesion mask is present, calculates:
     - The mean z-score within the lesion area (ipsilateral).
     - The mean z-score for the contralateral lesion area.
     - The mean z-score for the opposite hemisphere.

3. **Store Results**: Each subject’s calculated z-scores are stored in a structured list with columns for subject, feature, hemisphere, side (lesion or contralateral), and the z-score value.

4. **Save to CSV**: The compiled results are saved in a CSV file for further analysis.

This code ensures structured data for comparing lesion and contralateral hemisphere z-scores across subjects, facilitating analysis of asymmetries and inter/intra-z-scores.


In [25]:


# Read the list of subjects from the file
with open(subject_list_path, 'r') as file:
    subjects = [line.strip() for line in file if line.strip()]

def load_lesion_mask(subject, subject_dir):
    """
    Loads the lesion mask for the given subject from either the left or right hemisphere.
    Returns the lesion mask, lesion hemisphere, and contralateral hemisphere.
    """
    lesion_subject = subject.replace('_C_', '_T_') if '_C_' in subject else subject

    # Define paths for the lesion mask in both hemispheres
    lesion_mask_path_lh = os.path.join(subject_dir, lesion_subject, 'xhemi', 'surf_meld', 'lh.on_lh.lesion.mgh')
    lesion_mask_path_rh = os.path.join(subject_dir, lesion_subject, 'xhemi', 'surf_meld', 'rh.on_lh.lesion.mgh')

    if os.path.exists(lesion_mask_path_lh):
        lesion_hemi = 'lh'
        contralateral_hemi = 'rh'
        lesion_mask_path = lesion_mask_path_lh
    elif os.path.exists(lesion_mask_path_rh):
        lesion_hemi = 'rh'
        contralateral_hemi = 'lh'
        lesion_mask_path = lesion_mask_path_rh
    else:
        print(f"No valid lesion mask file found for {subject}")
        return None, None, None

    try:
        lesion_mask = nib.load(lesion_mask_path).get_fdata() > 0
        return lesion_mask, lesion_hemi, contralateral_hemi
    except Exception as e:
        print(f"Error loading lesion mask for {subject}: {e}")
        return None, None, None

def lesion_zscore(subject, subject_dir, feature_name):
    """
    Calculates the lesion z-scores for a specific feature of a subject.
    """
    z_scores = {'lesion': {}, 'lesion_contra': {}}

    # Load the lesion mask and determine hemispheres
    lesion_mask, lesion_hemi, contralateral_hemi = load_lesion_mask(subject, subject_dir)
    if lesion_mask is None:
        return None

    # Construct paths for z-scores based on feature names
    z_score_path_lesion = os.path.join(subject_dir, subject, f'{lesion_hemi}.inter_z.intra_z.{feature_name}.mgh')
    z_score_path_contra = os.path.join(subject_dir, subject, f'{contralateral_hemi}.inter_z.intra_z.{feature_name}.mgh')

    # Load z-scores for the lesion hemisphere and apply the lesion mask
    if os.path.exists(z_score_path_lesion):
        try:
            z_scores_lesion = nib.load(z_score_path_lesion).get_fdata()
            if z_scores_lesion.shape == lesion_mask.shape:
                lesion_z_score_values = z_scores_lesion[lesion_mask]
                z_scores['lesion'][lesion_hemi] = np.nanmean(lesion_z_score_values)
            else:
                print(f"Shape mismatch for {subject} in {lesion_hemi}: lesion mask {lesion_mask.shape}, z-scores {z_scores_lesion.shape}")
        except Exception as e:
            print(f"Error loading lesion z-score for {subject}: {e}")
    else:
        print(f"Lesion z-score file not found for {subject} in {lesion_hemi} with feature {feature_name} at path {z_score_path_lesion}")

    # Load z-scores for the contralateral hemisphere and apply the lesion mask
    if os.path.exists(z_score_path_contra):
        try:
            z_scores_contra = nib.load(z_score_path_contra).get_fdata()
            if z_scores_contra.shape == lesion_mask.shape:
                lesion_contra_z_score_values = z_scores_contra[lesion_mask]
                z_scores['lesion_contra'][contralateral_hemi] = np.nanmean(lesion_contra_z_score_values)
            else:
                print(f"Shape mismatch for {subject} in {contralateral_hemi}: lesion mask {lesion_mask.shape}, z-scores {z_scores_contra.shape}")
        except Exception as e:
            print(f"Error loading contralateral z-score for {subject}: {e}")
    else:
        print(f"Contralateral z-score file not found for {subject} in {contralateral_hemi} with feature {feature_name} at path {z_score_path_contra}")

    # Check if any scores were calculated
    return z_scores if z_scores['lesion'] or z_scores['lesion_contra'] else None

def process_subjects(subjects, subject_dir, feature_names):
    results = []
    for subject in subjects:
        for feature in feature_names:
            z_scores = lesion_zscore(subject, subject_dir, feature)
            if z_scores:
                for hemisphere, z_score in z_scores['lesion'].items():
                    results.append([subject, feature, hemisphere, 'lesion', z_score])
                for hemisphere, z_score in z_scores['lesion_contra'].items():
                    results.append([subject, feature, hemisphere, 'lesion_contra', z_score])
            else:
                print(f"No valid z-scores calculated for {subject} with feature {feature}")

    return results

# Define subject directory
subject_dir = '/Users/pierceburr/Documents/Documents/UCL_dissertation/meld/output/fastsurfer'

# Process subjects and save results to CSV
results = process_subjects(subjects, subject_dir, feature_names)

# Define the output directory and CSV file path
output_dir = os.path.join(subject_dir, 'inter.intra_z')
os.makedirs(output_dir, exist_ok=True)
csv_file_path = os.path.join(output_dir, 'results_inter_intra_z_with_contralateral_lesion_mask.csv')

# Write the results to a CSV file
with open(csv_file_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Subject', 'Feature', 'Hemisphere', 'Side', 'Z-Score'])
    writer.writerows(results)

# Output CSV path for easy access
print(f"Results saved to {csv_file_path}")



Results saved to /Users/pierceburr/Documents/Documents/UCL_dissertation/meld/output/fastsurfer/inter.intra_z/results_inter_intra_z_with_contralateral_lesion_mask.csv


In [26]:


def load_lesion_mask(subject, subject_dir):
    """
    Loads the lesion mask for the given subject from either the left or right hemisphere.
    Returns the lesion mask, lesion hemisphere, and contralateral hemisphere.
    """
    lesion_subject = subject.replace('_C_', '_T_') if '_C_' in subject else subject

    # Define paths for the lesion mask in both hemispheres
    lesion_mask_path_lh = os.path.join(subject_dir, lesion_subject, 'xhemi', 'surf_meld', 'lh.on_lh.lesion.mgh')
    lesion_mask_path_rh = os.path.join(subject_dir, lesion_subject, 'xhemi', 'surf_meld', 'rh.on_lh.lesion.mgh')

    if os.path.exists(lesion_mask_path_lh):
        lesion_hemi = 'lh'
        contralateral_hemi = 'rh'
        lesion_mask_path = lesion_mask_path_lh
    elif os.path.exists(lesion_mask_path_rh):
        lesion_hemi = 'rh'
        contralateral_hemi = 'lh'
        lesion_mask_path = lesion_mask_path_rh
    else:
        print(f"Error: No valid lesion mask file found for {subject}")
        return None, None, None

    try:
        lesion_mask = nib.load(lesion_mask_path).get_fdata() > 0
        return lesion_mask, lesion_hemi, contralateral_hemi
    except Exception as e:
        print(f"Error loading lesion mask for {subject}: {e}")
        return None, None, None

def lesion_zscore(subject, subject_dir, feature_name):
    """
    Calculates the lesion z-scores for a specific feature of a subject.
    """
    z_scores = {'lesion': None, 'lesion_contra': None}

    # Load the lesion mask and determine hemispheres
    lesion_mask, lesion_hemi, contralateral_hemi = load_lesion_mask(subject, subject_dir)
    if lesion_mask is None:
        return None

    # Construct paths for z-scores based on feature names
    z_score_path_lesion = os.path.join(subject_dir, subject, f'{lesion_hemi}.inter_z.intra_z.{feature_name}.mgh')
    z_score_path_contra = os.path.join(subject_dir, subject, f'{contralateral_hemi}.inter_z.intra_z.{feature_name}.mgh')

    # Load z-scores for the lesion hemisphere and apply the lesion mask
    if os.path.exists(z_score_path_lesion):
        try:
            z_scores_lesion = nib.load(z_score_path_lesion).get_fdata()
            if z_scores_lesion.shape == lesion_mask.shape:
                lesion_z_score_values = z_scores_lesion[lesion_mask]
                z_scores['lesion'] = lesion_z_score_values  # Keep the raw values
            else:
                print(f"Error: Shape mismatch for {subject} in {lesion_hemi}: lesion mask {lesion_mask.shape}, z-scores {z_scores_lesion.shape}")
        except Exception as e:
            print(f"Error loading lesion z-score for {subject}: {e}")
    else:
        print(f"Error: Lesion z-score file not found for {subject} in {lesion_hemi} with feature {feature_name} at path {z_score_path_lesion}")

    # Load z-scores for the contralateral hemisphere and apply the lesion mask
    if os.path.exists(z_score_path_contra):
        try:
            z_scores_contra = nib.load(z_score_path_contra).get_fdata()
            if z_scores_contra.shape == lesion_mask.shape:
                lesion_contra_z_score_values = z_scores_contra[lesion_mask]
                z_scores['lesion_contra'] = lesion_contra_z_score_values  # Keep the raw values
            else:
                print(f"Error: Shape mismatch for {subject} in {contralateral_hemi}: lesion mask {lesion_mask.shape}, z-scores {z_scores_contra.shape}")
        except Exception as e:
            print(f"Error loading contralateral z-score for {subject}: {e}")
    else:
        print(f"Error: Contralateral z-score file not found for {subject} in {contralateral_hemi} with feature {feature_name} at path {z_score_path_contra}")

    return z_scores if z_scores['lesion'] is not None or z_scores['lesion_contra'] is not None else None

def process_subjects(subjects, subject_dir, feature_names):
    results = []
    for subject in subjects:
        for feature in feature_names:
            z_scores = lesion_zscore(subject, subject_dir, feature)
            if z_scores:
                if z_scores['lesion'] is not None:
                    results.append([subject, feature, 'lesion', list(z_scores['lesion'])])
                if z_scores['lesion_contra'] is not None:
                    results.append([subject, feature, 'lesion_contra', list(z_scores['lesion_contra'])])
    return results


# Process subjects and save results to CSV
results = process_subjects(subjects, subject_dir, feature_names)

# Define the output directory and CSV file path
output_dir = os.path.join(subject_dir, 'inter.intra_z')
os.makedirs(output_dir, exist_ok=True)
csv_file_path = os.path.join(output_dir, 'results_inter_intra_z_with_contralateral_lesion_mask.csv')

# Write the results to a CSV file
with open(csv_file_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Subject', 'Feature', 'Side', 'Z-Scores'])
    for result in results:
        writer.writerow(result)

# Output CSV path for easy access
print(f"Results saved to {csv_file_path}")


Results saved to /Users/pierceburr/Documents/Documents/UCL_dissertation/meld/output/fastsurfer/inter.intra_z/results_inter_intra_z_with_contralateral_lesion_mask.csv


This script processes Z-score and age data for multiple subjects, combining, transforming, and aggregating it to create a structured dataset, which is saved to a CSV file. The steps are as follows:

1. **Load Data**: 
   - Loads Z-score data from a CSV file.
   - Loads age data from a separate CSV file.
   
2. **Clean Data**:
   - Strips any leading or trailing spaces from column names.
   - Ensures there are no extra spaces in the 'Subject' columns for accurate merging.

3. **Merge Data**:
   - Merges the age and Z-score data on the 'Subject' column, using a left join to keep all Z-score entries.

4. **Create Simplified Subject Identifier**:
   - Removes `_C_` and `_T_` substrings from the subject identifiers to create a common identifier (`Simplified_Subject`) for grouping.

5. **Separate Age and Z-Scores by Condition**:
   - Separates 'Age' and 'Z-Score' values into different columns based on whether the subject identifier contains `_C_` (Control) or `_T_` (Test).
   - Creates separate columns for lesion Z-scores, contralateral lesion Z-scores, and contralateral hemisphere Z-scores for each condition.

6. **Aggregate Data**:
   - Groups the data by the new `Simplified_Subject` and `Feature` columns.
   - Aggregates the separated values, using `max` to retain valid entries and avoid `None` values.
   
7. **Save Results**:
   - Outputs the aggregated DataFrame to a new CSV file for further analysis.
   
This script organizes data to support analyses comparing Z-scores between conditions, regions (lesion vs. contralateral), and hemispheres.


In [31]:
import pandas as pd
import numpy as np

# Load the Z-score data
z_scores_df = pd.read_csv(csv_file_path)

# Load the age data
age_df = pd.read_csv('pathtoagecsvfile/MELD_age.csv')

# Strip leading/trailing spaces from column names
z_scores_df.columns = z_scores_df.columns.str.strip()
age_df.columns = age_df.columns.str.strip()

# Strip spaces from text data in 'Subject' column if necessary
z_scores_df['Subject'] = z_scores_df['Subject'].str.strip()
age_df['Subject'] = age_df['Subject'].str.strip()

# Convert 'Z-Scores' from string representation of lists to actual lists
z_scores_df['Z-Scores'] = z_scores_df['Z-Scores'].apply(lambda x: np.fromstring(x.strip('[]'), sep=',') if pd.notnull(x) else np.nan)

# Take the mean of the Z-Scores lists for simplicity
z_scores_df['Z-Scores'] = z_scores_df['Z-Scores'].apply(lambda x: np.nanmean(x) if isinstance(x, np.ndarray) else np.nan)

# Merge the age DataFrame with the Z-score DataFrame on 'Subject'
merged_df = pd.merge(z_scores_df, age_df, on='Subject', how='left')

# Create a simplified subject identifier by removing '_C' and '_T'
merged_df['Simplified_Subject'] = merged_df['Subject'].str.replace(r'_C_|_T_', '_', regex=True)

# Separate columns for Age and Z-Score depending on '_C_' or '_T_'
merged_df['Age1'] = merged_df.apply(lambda x: x['Age'] if '_C_' in x['Subject'] else None, axis=1)
merged_df['Age2'] = merged_df.apply(lambda x: x['Age'] if '_T_' in x['Subject'] else None, axis=1)
merged_df['Lesion1'] = merged_df.apply(lambda x: x['Z-Scores'] if '_C_' in x['Subject'] and x['Side'] == 'lesion' else None, axis=1)
merged_df['Lesion2'] = merged_df.apply(lambda x: x['Z-Scores'] if '_T_' in x['Subject'] and x['Side'] == 'lesion' else None, axis=1)
merged_df['LesionContra1'] = merged_df.apply(lambda x: x['Z-Scores'] if '_C_' in x['Subject'] and x['Side'] == 'lesion_contra' else None, axis=1)
merged_df['LesionContra2'] = merged_df.apply(lambda x: x['Z-Scores'] if '_T_' in x['Subject'] and x['Side'] == 'lesion_contra' else None, axis=1)

# Group by the new subject identifier and the actual 'Feature' column, then aggregate the data
agg_df = merged_df.groupby(['Simplified_Subject', 'Feature']).agg({
    'Age1': 'max',  # Using max to skip None values, assuming only one valid entry per group
    'Age2': 'max',
    'Lesion1': 'max',
    'Lesion2': 'max',
    'LesionContra1': 'max',
    'LesionContra2': 'max'
}).reset_index()


# Save the aggregated DataFrame to a new CSV file
output_file_path = 'pathto/age_z_with_contralateral.csv'
agg_df.to_csv(output_file_path, index=False)

print(f"Aggregated data with lesion and contralateral lesion Z-scores saved to {output_file_path}")


Aggregated data with lesion and contralateral lesion Z-scores saved to /Users/pierceburr/Documents/Documents/UCL_dissertation/meld/output/fastsurfer/inter.intra_z/age_z_with_contralateral.csv
